# Data wrangling

In [1]:
import kagglehub
import pandas as pd
import numpy as np
from kagglehub import KaggleDatasetAdapter

## 1. Download disaster data from Kaggle

In [2]:
# Load a DataFrame with a specific version of a CSV
raw_data_df=kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    'headsortails/us-natural-disaster-declarations',
    'us_disaster_declarations.csv',
)

In [3]:
raw_data_df.head().transpose()

,0,1,2,3,4
fema_declaration_string,DR-1-GA,DR-2-TX,DR-3-LA,DR-4-MI,DR-5-MT
disaster_number,1,2,3,4,5
state,GA,TX,LA,MI,MT
declaration_type,DR,DR,DR,DR,DR
declaration_date,1953-05-02T00:00:00Z,1953-05-15T00:00:00Z,1953-05-29T00:00:00Z,1953-06-02T00:00:00Z,1953-06-06T00:00:00Z
fy_declared,1953,1953,1953,1953,1953
incident_type,Tornado,Tornado,Flood,Tornado,Flood
declaration_title,Tornado,Tornado & Heavy Rainfall,Flood,Tornado,Floods
ih_program_declared,0,0,0,0,0
ia_program_declared,1,1,1,1,1


In [4]:
raw_data_df.to_parquet('../data/raw_disaster_data.parquet')

## 2. Download FIPS code database from census.gov

In [5]:
fips_df=pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt', sep='|')
fips_df.head()

,STATE,STATEFP,COUNTYFP,COUNTYNS,COUNTYNAME,CLASSFP,FUNCSTAT
0,AL,1,1,161526,Autauga County,H1,A
1,AL,1,3,161527,Baldwin County,H1,A
2,AL,1,5,161528,Barbour County,H1,A
3,AL,1,7,161529,Bibb County,H1,A
4,AL,1,9,161530,Blount County,H1,A


In [6]:
# Save the raw data
fips_df.to_parquet('../data/fips_codes.parquet')

## 3. Add county name based on FIPS code

To add the county name, we need to concatenate the `STATEFP` and `COUNTYFP` columns in the census data, then use that string to translate between the `fips` column in the disaster data and the `COUNTYNAME` column in the census data. Let's put the logic in a function for easy refactoring.

In [7]:
def decode_fips(fips_df: pd.DataFrame, raw_data_df: pd.DataFrame) -> pd.DataFrame:
    '''Takes census.gov FIPS dataframe and disaster dataframe, adds human readable 
    county column to disaster data using the census data as look-up table returns 
    updated disaster dataframe'''

    # First extract the state and county FIPS codes
    state_fp=fips_df['STATEFP'].to_list()
    county_fp=fips_df['COUNTYFP'].to_list()

    # Left zero pad state and county FIPS codes to two and three digits respectively
    state_fp=[str(n).zfill(2) for n in state_fp]
    county_fp=[str(n).zfill(3) for n in county_fp]

    # Concatenate the state and county codes to get the full FIPS code
    fips=[i+j for i,j in zip(state_fp, county_fp)]

    # Make a dictionary to translate FIPS county codes to county names
    fips_lookup=dict(zip(fips, fips_df['COUNTYNAME']))

    # Add a new column to the raw disaster data containing the FIPS code to be translated
    data_df=raw_data_df.copy()
    data_df['county_name']=raw_data_df['fips'].apply(str)

    # Translate the column values from FIPS to county name, using 'Unknown' for County if we don't
    # have the FIPS code in our dict
    data_df['county_name']=data_df['county_name'].map(fips_lookup).fillna('Unknown')

    return data_df

In [8]:
# Do the decoding
data_df=decode_fips(fips_df, raw_data_df)

## 4. Save the data

In [9]:
# Take a look at what we have
data_df.tail().transpose()

,64087,64088,64089,64090,64091
fema_declaration_string,DR-4696-ME,DR-4697-MS,DR-4697-MS,DR-4697-MS,DR-4697-MS
disaster_number,4696,4697,4697,4697,4697
state,ME,MS,MS,MS,MS
declaration_type,DR,DR,DR,DR,DR
declaration_date,2023-03-22T00:00:00Z,2023-03-26T00:00:00Z,2023-03-26T00:00:00Z,2023-03-26T00:00:00Z,2023-03-26T00:00:00Z
fy_declared,2023,2023,2023,2023,2023
incident_type,Severe Storm,Severe Storm,Severe Storm,Severe Storm,Severe Storm
declaration_title,Severe Storm And Flooding,"Severe Storms, Straight-Line Winds, And Tornadoes","Severe Storms, Straight-Line Winds, And Tornadoes","Severe Storms, Straight-Line Winds, And Tornadoes","Severe Storms, Straight-Line Winds, And Tornadoes"
ih_program_declared,0,1,1,1,1
ia_program_declared,0,0,0,0,0


In [10]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64092 entries, 0 to 64091
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   fema_declaration_string     64092 non-null  object
 1   disaster_number             64092 non-null  int64 
 2   state                       64092 non-null  object
 3   declaration_type            64092 non-null  object
 4   declaration_date            64092 non-null  object
 5   fy_declared                 64092 non-null  int64 
 6   incident_type               64092 non-null  object
 7   declaration_title           64092 non-null  object
 8   ih_program_declared         64092 non-null  int64 
 9   ia_program_declared         64092 non-null  int64 
 10  pa_program_declared         64092 non-null  int64 
 11  hm_program_declared         64092 non-null  int64 
 12  incident_begin_date         64092 non-null  object
 13  incident_end_date           55682 non-null  ob

In [11]:
# Save the data
data_df.to_parquet('../data/master_disaster_data.parquet')

## 5. Data shape formatting

Now the real work begins - we need to set this dataset up for modeling as a multi-label time-series prediction problem. To make our lives a little easier, we will treat each county as an independent sample (this is likely not strictly true, as especially weather based disasters may tend to co-occur geographically). Doing so increases the number of observations and simplifies the input. Each individual time point will be a vector of dummy encoded disasters, including one feature for 'no-disaster'. This will give is n different time series for the n different counties. Care will need to be taken when splitting/sampling and generating batches not to cross between the time series from different counties.

Let's dig in!

### 5.1. Feature selection

In [12]:
# Get only the features we are going to work with into a new dataframe
working_df=data_df[['incident_begin_date','fips','incident_type']].copy()

# Convert 'incident_begin_date' to month and year columns and set as index
working_df['incident_begin_date']=pd.to_datetime(working_df['incident_begin_date'])

# Remove duplicates
working_df=working_df.drop_duplicates(keep='first')

# Fix the index
working_df.reset_index(inplace=True, drop=True)
working_df.head()

,incident_begin_date,fips,incident_type
0,1953-05-02 00:00:00+00:00,13000,Tornado
1,1953-05-15 00:00:00+00:00,48000,Tornado
2,1953-05-29 00:00:00+00:00,22000,Flood
3,1953-06-02 00:00:00+00:00,26000,Tornado
4,1953-06-06 00:00:00+00:00,30000,Flood


### 5.2. Disaster encoding

In [13]:
# Encode the disaster types to dummies
disaster_dummies=pd.get_dummies(working_df['incident_type'], dtype=int)

# Drop some disaster types - the dataset is big enough that it causes out of memory
# crashes in the Codespace. But more than that, we have some disaster types
# the are rare/hard to predict. Let's focus on the wether based types and get
# rid of the rest
disaster_drops=['Biological', 'Chemical', 'Earthquake', 'Fishing Losses', 'Human Cause', 'Other', 'Terrorist', 'Toxic Substances', 'Volcanic Eruption']
disaster_dummies.drop(disaster_drops, axis=1, inplace=True)

# We also have 4 disaster types that are winter weather relates, let's combine those
disaster_dummies['Winter weather']=disaster_dummies['Severe Ice Storm'] + disaster_dummies['Snowstorm'] + disaster_dummies['Freezing'] + disaster_dummies['Winter Storm']
disaster_dummies.drop(['Severe Ice Storm','Snowstorm','Freezing','Winter Storm'], axis=1, inplace=True)
disaster_dummies.head()

,Coastal Storm,Dam/Levee Break,Drought,Fire,Flood,Hurricane,Mud/Landslide,Severe Storm,Tornado,Tropical Storm,Tsunami,Typhoon,Winter weather
0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0


In [14]:
# Combine the disaster dummies and the original dataframe and remove the 'incident_type' column
working_df=pd.concat([working_df.reset_index(drop=True), disaster_dummies.reset_index(drop=True)], axis=1)
working_df.drop('incident_type', axis=1, inplace=True)
working_df.head()

,incident_begin_date,fips,Coastal Storm,Dam/Levee Break,Drought,Fire,Flood,Hurricane,Mud/Landslide,Severe Storm,Tornado,Tropical Storm,Tsunami,Typhoon,Winter weather
0,1953-05-02 00:00:00+00:00,13000,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1953-05-15 00:00:00+00:00,48000,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1953-05-29 00:00:00+00:00,22000,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1953-06-02 00:00:00+00:00,26000,0,0,0,0,0,0,0,0,1,0,0,0,0
4,1953-06-06 00:00:00+00:00,30000,0,0,0,0,1,0,0,0,0,0,0,0,0


Now, we need to regularize the time series to a frequency of months across the span of years within each county.

### 5.3. Time series regularization

In [15]:
def sum_months(group: pd.DataFrame) -> pd.DataFrame:
    '''Takes a yearly groupby object and sums features over months'''

    group=group.resample('ME').sum()

    return group


def resample_months(group: pd.DataFrame) -> pd.DataFrame:
    '''Takes working dataframe and resamples frequency to months.
    Returns updated dataframe'''

    # Set 'incident_begin_date' as datetime axis
    group=group.set_index('incident_begin_date')

    # Sum the disasters in each month by year. This removes duplicates where
    # there was more than one disaster in a month.
    group=group.groupby(group.index.year, group_keys=False).apply(sum_months)

    # Resample to monthly frequency
    group=group.resample('D').asfreq()

    # Fill missing values with 0
    group=group.fillna(0)

    # Convert everything to int
    group=group.astype(bool)

    # Reset the index, preserving the `incident_begin_date`
    group.reset_index(inplace=True, drop=False)

    return group

# Do the resampling
resampled_working_df=working_df.groupby('fips', group_keys=True).apply(resample_months, include_groups=False)

### 5.4. Clean up and finalize features

In [16]:
# Clean up the index
resampled_working_df.reset_index(inplace=True)
resampled_working_df.drop('level_1', axis=1, inplace=True)

In [17]:
# Fix the FIPS data type
resampled_working_df['fips']=resampled_working_df['fips'].astype(np.int32)

In [18]:
# Extract month and year from 'incident_begin_date' and drop
resampled_working_df['year']=resampled_working_df['incident_begin_date'].dt.year
resampled_working_df['month']=resampled_working_df['incident_begin_date'].dt.month
resampled_working_df.drop('incident_begin_date', axis=1, inplace=True)
resampled_working_df.head()

,fips,Coastal Storm,Dam/Levee Break,Drought,Fire,Flood,Hurricane,Mud/Landslide,Severe Storm,Tornado,Tropical Storm,Tsunami,Typhoon,Winter weather,year,month
0,1000,False,False,False,False,True,False,False,False,False,False,False,False,False,1961,2
1,1000,False,False,False,False,False,False,False,False,False,False,False,False,False,1961,3
2,1000,False,False,False,False,False,False,False,False,False,False,False,False,False,1961,3
3,1000,False,False,False,False,False,False,False,False,False,False,False,False,False,1961,3
4,1000,False,False,False,False,False,False,False,False,False,False,False,False,False,1961,3


In [19]:
resampled_working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55901655 entries, 0 to 55901654
Data columns (total 16 columns):
 #   Column           Dtype
---  ------           -----
 0   fips             int32
 1   Coastal Storm    bool 
 2   Dam/Levee Break  bool 
 3   Drought          bool 
 4   Fire             bool 
 5   Flood            bool 
 6   Hurricane        bool 
 7   Mud/Landslide    bool 
 8   Severe Storm     bool 
 9   Tornado          bool 
 10  Tropical Storm   bool 
 11  Tsunami          bool 
 12  Typhoon          bool 
 13  Winter weather   bool 
 14  year             int32
 15  month            int32
dtypes: bool(13), int32(3)
memory usage: 1.3 GB


Note: the time series resampling uses more than 20 GB of memory, and the final dataframe occupies more than 6 GB with int64 dtypes for the disaster features. Using int32 brings it down to 3.5 GB in memory, but the resample still takes almost 20 GB. Using bool for the disaster features gets the memory occupancy under 2 GB, but the lift to resample it is still around 18 GB.

## 6. Save the resampled data

### 6.1. Complete dataset

In [20]:
# Save the complete dataset
resampled_working_df.to_parquet('../data/resampled_disaster_data_all.parquet')

### 6.2. Most recent 5 years

In [22]:
# Also save a subset from the last 5 years
sample_resampled_working_df=resampled_working_df[resampled_working_df['year'] >= 1998]
sample_resampled_working_df.reset_index(inplace=True, drop=True)
sample_resampled_working_df.to_parquet('../data/resampled_disaster_data_1998-current.parquet')
sample_resampled_working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27375905 entries, 0 to 27375904
Data columns (total 16 columns):
 #   Column           Dtype
---  ------           -----
 0   fips             int32
 1   Coastal Storm    bool 
 2   Dam/Levee Break  bool 
 3   Drought          bool 
 4   Fire             bool 
 5   Flood            bool 
 6   Hurricane        bool 
 7   Mud/Landslide    bool 
 8   Severe Storm     bool 
 9   Tornado          bool 
 10  Tropical Storm   bool 
 11  Tsunami          bool 
 12  Typhoon          bool 
 13  Winter weather   bool 
 14  year             int32
 15  month            int32
dtypes: bool(13), int32(3)
memory usage: 652.7 MB
